In [1]:
from experimental.enc_dec_dataset import EncDecDataset, CustomEncoder
from experimental.enc_dec_training import EncDecTrainer
from transformers import EncoderDecoderModel, AutoTokenizer

import torch
import json
import os
import os.path as op

model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16", pad_token_id=0)
#orig_model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16", pad_token_id=0)
tokenizer = AutoTokenizer.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16", pad_token_id=0)
cust_enc = CustomEncoder(model.encoder)
model.encoder = cust_enc
#cust_dec = CustomDecoder(model.decoder)

model.encoder = cust_enc


DATA_DIR = "DialogSum_Data"

SAMPLE_DATA = op.join(DATA_DIR, 'dialogsum.sample.jsonl')
TEST_DATA = op.join(DATA_DIR, 'dialogsum.test.jsonl')
TRAIN_DATA = op.join(DATA_DIR, 'dialogsum.train.jsonl')
DEV_DATA = op.join(DATA_DIR, 'dialogsum.dev.jsonl')

def load_jsonl(filepath):
    
    output_list = []
    
    with open(filepath) as sd_file:
        lines = sd_file.readlines()
    
    for line in lines:
        output_list.append(json.loads(line))

    return output_list


Some weights of the model checkpoint at patrickvonplaten/bert2bert-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.bert.pooler.dense.weight', 'decoder.bert.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
dialogue = ["this is a sentence", "what is the sentence about", "yes it is"]
tokens = tokenizer.batch_encode_plus(dialogue, padding=True, return_tensors="pt")



tokens.pop("attention_mask")
tokens["attention_mask"] = torch.Tensor([[1,1,1]])
tokens.pop("token_type_ids")
output = model.generate(**tokens)
print(tokenizer.batch_decode(output))



inputs:
tensor([[ 101, 2023, 2003, 1037, 6251,  102,    0]])
torch.Size([1, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
inputs:
tensor([[ 101, 2054, 2003, 1996, 6251, 2055,  102]])
torch.Size([1, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
attention_scores:
torch.Size([1, 12, 7, 7])
atte

['[CLS] the incident took place in the town of kew, near the capital of south africa. the town is a popular destination for the local population. the city has a number of famous actors and actresses. the actor is now a celebrity in the uk. he is the latest in a series of celebrity celebrity - themed films. [SEP]',
 '[CLS] the incident took place in the town of kew, near the capital of south africa. the town is a popular destination for the local population. the city has a number of famous actors and actresses. the actor is now a celebrity in the uk. he is the latest in a series of celebrity celebrity - themed films. [SEP]',
 '[CLS] the incident took place in the town of kew, near the capital of south africa. the town is a popular destination for the local population. the city has a number of famous actors and actresses. the actor is now a celebrity in the uk. he is the latest in a series of celebrity celebrity - themed films. [SEP]']

In [ ]:


if __name__ == "__main__":




    model_params = {
        "TRAIN_BATCH_SIZE": 1,  # training batch size
        "VALID_BATCH_SIZE": 1,  # validation batch size
        "TRAIN_EPOCHS":  1,  # number of training epochs
        "VAL_EPOCHS": 1,  # number of validation epochs
        "LEARNING_RATE": 1e-3,  # learning rate
        "EXPERIMENT_NAME": "preliminary",
        #"MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
        #"MAX_TARGET_TEXT_LENGTH": 50,  # max length of target text
        "SEED": 42,  # set seed for reproducibility
        }


    dev_data_list = load_jsonl(DEV_DATA)
    dev_dataset = EncDecDataset(dev_data_list, 
                  tokenizer=tokenizer)

    train_data_list = load_jsonl(TRAIN_DATA)
    train_dataset = EncDecDataset(train_data_list,
                        tokenizer=tokenizer)

    model = EncDecTrainer(train_dataset=train_dataset, val_dataset=dev_dataset,
                    model=model, tokenizer=tokenizer, model_params=model_params)
                  